In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=2, threads_per_worker=2,dashboard_address=':8787')

client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:40177 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 4 Memory: 35.01 GB


In [2]:
## Path for modules

import sys

import numpy as np
import xarray as xr

from netCDF4 import Dataset

sys.path.insert(0,"/mnt/meom/workdir/henelle/Notebooks/git/xscale")
import xscale
import xscale.spectral.fft as xfft

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy.ma as ma

import matplotlib.cm as mplcm
import matplotlib.colors as colors
from matplotlib import gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import date, datetime
from xhistogram.xarray import histogram

import pandas as pd

import seaborn as sns
sns.set(color_codes=True)

%matplotlib inline

In [3]:
def curl(u,v,e1v,e2u,ff):
    '''
    This routine computes the relative vorticity from 2D fields of horizontal velocities and the spatial Coriolis parameter.
    '''
    #Computation of dy(u)
    fe2u=1/e2u
    fse2u=fe2u.squeeze()
    dyu=(u.shift(y=-1) - u)*fse2u
    #Computation of dx(v)
    fe1v=1/e1v
    fse1v=fe1v.squeeze()
    dxv=(v.shift(x=-1) - v)*fse1v
    #Projection on the grid T
    dxvt=0.5*(dxv.shift(y=-1)+dxv)
    dyut=0.5*(dyu.shift(x=-1)+dyu)
    #Computation of the vorticity divided by f
    fff=1/ff
    curl=(dxvt-dyut)*fff
    return curl

In [4]:
def strain(u,v,e1u,e1v,e2u,e2v,ff):
    '''
    This routine computes the relative vorticity from 2D fields of horizontal velocities and the spatial Coriolis parameter.
    '''
    #Computation of dy(u)
    fe2u=1/e2u
    fse2u=fe2u.squeeze()
    dyu=(u.shift(y=-1) - u)*fse2u
    #Computation of dx(v)
    fe1v=1/e1v
    fse1v=fe1v.squeeze()
    dxv=(v.shift(x=-1) - v)*fse1v
    #Computation of dy(v)
    fe2v=1/e2v
    fse2v=fe2v.squeeze()
    dyv=(v.shift(y=-1) - v)*fse2v
    #Computation of dx(u)
    fe1u=1/e1u
    fse1u=fe1u.squeeze()
    dxu=(u.shift(x=-1) - u)*fse1u
    #Projection on the grid T
    dxvt=0.5*(dxv.shift(y=-1)+dxv)
    dyut=0.5*(dyu.shift(x=-1)+dyu)
    dxut=0.5*(dxu.shift(x=-1)+dxu)
    dyvt=0.5*(dyv.shift(y=-1)+dyv)
    #Computation of the strain divided by f
    fff=1/ff
    strain=np.sqrt( (dyut+dxvt)*(dyut+dxvt) + (dxut-dyvt)*(dxut-dyvt) ) *fff
    return strain

In [5]:
## Dataset : grid files and one example of U and V for test


# Opening grid files

dirgrid='/mnt/meom/workdir/alberta/eNATL60/eNATL60-I/'
gridfile=dirgrid+'mesh_hgr_eNATL60ACO_3.6.nc'
dsgrid=xr.open_dataset(gridfile,chunks={'x':200,'y':200})

e1u=dsgrid.e1u
e1v=dsgrid.e1v
e2u=dsgrid.e2u
e2v=dsgrid.e2v
ff=dsgrid.ff



# Opening u & v grid files
# With tide

dirvarT0='/mnt/meom/workdir/alberta/eNATL60/eNATL60-BLBT02-S/1h/ACO/'
ufileT0=dirvarT0+'eNATL60ACO-BLBT02_y2009m*.1h_vozocrtx10m.nc' # JAS
vfileT0=dirvarT0+'eNATL60ACO-BLBT02_y2009m*.1h_vomecrty10m.nc' # JAS
dsuT0=xr.open_mfdataset(ufileT0,combine='by_coords',parallel=True,chunks={'x':200,'y':200})
dsvT0=xr.open_mfdataset(vfileT0,combine='by_coords',parallel=True,chunks={'x':200,'y':200})
uT0=dsuT0.vozocrtx
vT0=dsvT0.vomecrty 
lon=dsuT0.nav_lon
lat=dsuT0.nav_lat

# Without tide

dirvar00='/mnt/meom/workdir/alberta/eNATL60/eNATL60-BLB002-S/1h/ACO/'
ufile00=dirvar00+'eNATL60ACO-BLB002_y2009m*.1h_vozocrtx10m.nc' # JAS
vfile00=dirvar00+'eNATL60ACO-BLB002_y2009m*.1h_vomecrty10m.nc' # JAS
dsu00=xr.open_mfdataset(ufile00,combine='by_coords',parallel=True,chunks={'x':200,'y':200})
dsv00=xr.open_mfdataset(vfile00,combine='by_coords',parallel=True,chunks={'x':200,'y':200})
u00=dsu00.vozocrtx
v00=dsv00.vomecrty

In [6]:
# Filtering of u & v

T=2*np.pi/(1E-4) # Coriolis period

wuT0 = uT0.window
wuT0.set(n=48,dim='time_counter', cutoff=2*T)
uT0_filt = wuT0.convolve()

wu00 = u00.window
wu00.set(n=48,dim='time_counter', cutoff=2*T)
u00_filt = wu00.convolve()

wvT0 = vT0.window
wvT0.set(n=48,dim='time_counter', cutoff=2*T)
vT0_filt = wvT0.convolve()

wv00 = v00.window
wv00.set(n=48,dim='time_counter', cutoff=2*T)
v00_filt = wv00.convolve()


In [7]:
### Compute curl and strain with python

curlT0_filt   = curl(uT0_filt,vT0_filt,e1v,e2u,ff)
curl00_filt   = curl(u00_filt,v00_filt,e1v,e2u,ff)
strainT0_filt = strain(uT0_filt,vT0_filt,e1u,e1v,e2u,e2v,ff)
strain00_filt = strain(u00_filt,v00_filt,e1u,e1v,e2u,e2v,ff)


In [10]:
curlT0_filt.squeeze()

<xarray.DataArray (time_counter: 2208, y: 1191, x: 781)>
dask.array<getitem, shape=(2208, 1191, 781), dtype=float64, chunksize=(24, 200, 200), chunktype=numpy.ndarray>
Coordinates:
    depthv         float32 9.318418
    time_centered  (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-09-30T23:30:00
  * time_counter   (time_counter) datetime64[ns] 2009-07-01T00:30:00 ... 2009-09-30T23:30:00
    depthu         float32 9.318418
Dimensions without coordinates: y, x

In [ ]:
curlT0_filt_da=xr.DataArray(curlT0_filt.squeeze(),dims=['time_counter','y','x'],name="Curl_Tide_Filt",coords=['lon','lat'])
curlT0_filt_da.attrs['Name']='eNATL60ACO-BLBT02_y2009m07-09_socurloverf10m_filt2T.nc'
curlT0_filt_da.to_dataset().to_netcdf(path='/mnt/meom/workdir/alberta/eNATL60/eNATL60-BLBT02-S/1h/ACO/eNATL60ACO-BLBT02_y2009m07-09_socurloverf10m_filt2T.nc',mode='w',engine='scipy')


distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker exceeded 95% memory budget. Restarting
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Worker process still alive after 3 seconds, killing


In [ ]:
KE_JFM_wavenum_freq_spectrum_da = xr.DataArray(KE_JFM_wavenum_freq_spectrum,dims=['frequency','wavenumber'],name="Ke_spectrum",coords=[frequency_JFM ,wavenumber_JFM])
KE_JFM_wavenum_freq_spectrum_da.attrs['Name'] = 'KE_Spectrum_JFM_w_k_from_1h_eNATL60-BLBT02.nc'

KE_JFM_wavenum_freq_spectrum_da.to_dataset().to_netcdf(path='KE_Spectrum_JFM_w_k_from_1h_eNATL60-BLBT02.nc',mode='w',engine='scipy')
